# Similaridade de texto (palavras)

A similaridade de texto é utilizada para comparar palavras similares (que foram escritas de forma diferente).
Alguns exemplos:

* `São Paulo`
* `Sao Paulo`
*  `São Paulo   ` (tem espaço no final)
* `são paulo`

Todas as palavras acima se referem a cidade de São Paulo, mas se efetuar umas simples comparação de _strings_ (`str1 == str2`) o resultado será `False`.

É possível tratar esses casos, como (`str.strip()`) para remover espaços no começo/final do texto.
Outro detalhe é `str.lower()` ou `str.upper()` para manter tudo minúsculo/maiúsculo.
Finalmente, é possível remover toda a acentuação para evitar problemas.

Entretanto, essa não é uma boa adordagem (principalmente remover acentuação, as outras duas podem ser boas ideias) visto que estamos alterando o texto para encontrar o nome `São Paulo`.

Uma alternativa é computar um número (entre 0 e 100, por exemplo) que indica o quão similares são duas palavras.
Sendo que, 0 indica palavras completamente diferentes e 100 indica que as palavras são idênticas.
Assim, podemos definir um limiar de forma que, qualquer similaridade acima de 80 aceitamos como algo válido.
O valor de limiar vai variar com o contexto, por isso é importante testar várias abordagens.

Existem várias métricas para calcular essa similaridade: https://en.wikipedia.org/wiki/Edit_distance

Vamos trabalhar com a distância de Levenshtein: https://en.wikipedia.org/wiki/Levenshtein_distance

A distância de Levenshtein calcula o número minímino de alterações (inserir, remover, substituir) que devemos fazer em uma string para que ela fique igual a outra.

Tutorial `fuzzywuzzy` https://www.datacamp.com/community/tutorials/fuzzy-string-python

Outra referência é a seção 3.3.3 na página 58 do livro [_An Introduction to Information Retrieval_](https://nlp.stanford.edu/IR-book/information-retrieval-book.html)

In [ ]:
# distância de levenshtein
import numpy as np

def levenshtein(a, b):
    pass

assert levenshtein('oi', 'oi') == 0
assert levenshtein('car', 'cat') == 1
assert levenshtein('oi', 'ola') == 2
assert levenshtein('cats', 'fast') == 3
assert levenshtein('sitting', 'kitten') == 3
assert levenshtein('sunday', 'saturday') == 3

In [4]:
from fuzzywuzzy import fuzz

/home/julio/Projects/pos-furb-recuperacao/.venv/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
fuzz.ratio('São Paulo', 'Sao Paulo')

89

In [6]:
fuzz.ratio('São Paulo', 'sao paulo')

67

In [7]:
fuzz.ratio('São Paulo', 'São Paulo    ')

82

In [8]:
fuzz.ratio('Sao Paulo', 'São Paulo')

89

Assim, podemos ter uma lista de cidades e ver quais pessoas moram em `São Paulo`

In [10]:
pessoas = [
    {'nome': 'Pessoa A', 'cidade': 'São Paulo'},
    {'nome': 'Pessoa B', 'cidade': ' São Paulo '},
    {'nome': 'Pessoa C', 'cidade': 'Sao Paulo'},
    {'nome': 'Pessoa D', 'cidade': 'sao paulo'},
    {'nome': 'Pessoa E', 'cidade': 'SaoPaulo'},
    {'nome': 'Pessoa F', 'cidade': 'S. Paulo'},
    {'nome': 'Pessoa G', 'cidade': 'Blumenau'},
    {'nome': 'Pessoa H', 'cidade': 'Pomerode'},
    {'nome': 'Pessoa I', 'cidade': 'Rio de Janeiro'},
]

scores = [fuzz.ratio('São Paulo', p['cidade']) for p in pessoas]
scores

[100, 90, 89, 67, 82, 82, 24, 24, 35]

Entretanto, nem sempre é possível comparar o texto diretamente.
Por exemplo, o nome da cidade pode estar no meio de uma frase.
Para isso, é possível fazer uma comparação parcial.

`fuzz.ratio()` compara dois textos por completo, com a similaridade entre eles.

`fuzz.partial_ratio()` faz a comparação em substrings.

In [13]:
fuzz.ratio('São Paulo', 'João mudou-se para São Paulo em 2019')

40

In [11]:
fuzz.partial_ratio('São Paulo', 'João mudou-se para São Paulo em 2019')

100

In [12]:
fuzz.partial_ratio('São Paulo', 'João mudou-se para Sao Paulo em 2019')

89

Outro caso comum, é a ordem dos textos.

Por exemplo, `Comparamos o Dispositivo A com o Dispositivo B` e `Comparamos o Dispositivo B com o Dispositivo A`.

In [14]:
fuzz.partial_ratio('Dispositivo A com Dispositivo B', 'Comparamos Dispositivo A com Dispositivo B')

100

In [17]:
fuzz.partial_ratio('Dispositivo A com Dispositivo B', 'Comparamos dispositivo B com dispositivo A')

55

In [18]:
fuzz.token_sort_ratio('Dispositivo A com Dispositivo B', 'Comparamos dispositivo B com dispositivo A')

85

Outro caso é o uso da comparação `Dispositivo A com Dispositivo B` ou `Dispositivo A x Dispositivo B`.

In [19]:
fuzz.token_sort_ratio('Dispositivo A x Dispositivo B', 'Comparamos dispositivo B com dispositivo A')

76

In [21]:
fuzz.token_set_ratio('Dispositivo A x Dispositivo B', 'Comparamos dispositivo B com dispositivo A')

94

## Processamento de linguagem natural

São técnicas utilizadas para trabalhar com texto que englobam desde _tokenização_ (separar uma string em palavras) até a tradução de texto.

A similaridade de palavras acima demonstra um uso de processamento de linguagem natural.

Nesta parte, vamos ver alguns conceitos básicos utilizando as bibliotecas `NLTK` e `spaCy`.

Uma comparação entre `NLTK` e `spaCy` https://medium.com/@akankshamalhotra24/introduction-to-libraries-of-nlp-in-python-nltk-vs-spacy-42d7b2f128f2

In [1]:
# Fonte http://www.furb.br/web/1704/noticias/furb-esta-entre-4-melhores-de-sc-em-ranking-da-america-latina/7944
texto = '''O Brasil desponta com as melhores Universidades da América Latina, com o total de 3 no top 10,
seguida do Chile, Colômbia e México com 2 em cada país, além da Argentina com uma.
No primeiro lugar está a Pontifícia Universidade Católica do Chile, seguida da Universidade de São Paulo (USP).
'''

## Tokenização

Separara um texto em uma lista de palavras.
Note que a forma mais simples de resolver esse problema é `texto.split(' ')` apenas separando por espaços.
Entretanto, essa abordagem falha em um texto como `... fim do dia.`, onde `dia.` seria um token, mas deveria ser separdo em dois: `dia` e `.`.
O mesmo acontece para nomes, `João B. da Silva`, é importante notar que `B.` não é o final da frase, mas uma abreviação.

In [2]:
from nltk.tokenize import word_tokenize

In [3]:
# Pode ser necessário executar
# import nltk
# nltk.download('punkt')

In [4]:
tokens = word_tokenize(texto)
for (i, token) in enumerate(tokens):
    print(token)
    if i == 10:
        break

O
Brasil
desponta
com
as
melhores
Universidades
da
América
Latina
,


In [5]:
import spacy

ModuleNotFoundError: No module named 'spacy'

In [6]:
# pode ser necessário instalar o modelo
# python -m spacy download pt_core_news_sm
nlp = spacy.load('pt_core_news_sm')

NameError: name 'spacy' is not defined

In [7]:
doc = nlp(texto)

NameError: name 'nlp' is not defined

In [8]:
for (i, token) in enumerate(doc):
    print(token.text)
    if i == 10:
        break

NameError: name 'doc' is not defined

In [9]:
# apenas os verbos
verbos = [token.text for token in doc if token.pos_ == 'VERB']
verbos

NameError: name 'doc' is not defined

## Remoção de _stopwords_

_Stopwords_ são palavras utilizadas para conectar texto (`de`, `a`, `e`, outros) e não estão ligadas diretamente ao sentido da frase.

In [10]:
from nltk.corpus import stopwords

In [11]:
# pode ser necessário executar
# import nltk
# nltk.download('stopwords')

In [12]:
sws = stopwords.words('portuguese')
sws[:10]

['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um']

In [13]:
tokens_sem_stopwords = [t for t in tokens if t not in sws]
print(len(tokens), len(tokens_sem_stopwords))
tokens_sem_stopwords[:10]

60 41


['O',
 'Brasil',
 'desponta',
 'melhores',
 'Universidades',
 'América',
 'Latina',
 ',',
 'total',
 '3']

In [51]:
spacy_tokens_sem_stopwords = [t for t in doc if not t.is_stop]
spacy_tokens_sem_stopwords[:10]

[O, Brasil, desponta, melhores, Universidades, América, Latina, ,, o, total]

## Stem

_Stem_ é um método para reduzir uma palavra ao radical/raiz.
Como os verbos são conjugados, pode ser difícil trabalhar com todas as variações.
Uma forma de reduzir esse problema, é trabalhar apenas com o radical do verbo.
Por: `copiar` -> `copi`

In [35]:
# pode ser necessário executar
# import nltk
# nltk.download('rslp')

In [36]:
stemmer = nltk.stem.RSLPStemmer()

In [44]:
stemmer.stem("necessário")

'necess'

In [46]:
stemmer.stem("necessidade")

'necess'

In [45]:
stemmer.stem("desnecessário")

'desnecess'

In [43]:
tokens_stem = [stemmer.stem(t) for t in tokens_sem_stopwords]
tokens_stem[:10]

['o',
 'brasil',
 'despont',
 'melhor',
 'univers',
 'amér',
 'latin',
 ',',
 'total',
 '3']

## Lemmatization

Enquanto _stemming_ reduz as palavras removendo partes no começo e final, _lemmatization_ reduz a palavra na forma do dicionário.

Para mais informações: https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html

In [54]:
lemmas = [t.lemma_ for t in spacy_tokens_sem_stopwords]
lemmas[:10]

['O',
 'Brasil',
 'despontar',
 'melhorar',
 'Universidades',
 'América',
 'Latina',
 ',',
 'o',
 'total']

## TF-IDF

_Term Frequency - Inverse Document Frequency_ é uma forma de calcular a importância de uma palavra em um documento.

### Term Frequency

Quantas vezes um termo (palavra) aparece em um documento.

* $\text{TF}_{t, d} = f_{t, d}$

Sendo $t$ um termo e $d$ um documento, ou seja, $f_{t, d}$ é simplesmente a contagem do termo $t$ no documento $d$.

O problema com a definição anterior é que determinados termos podem aparecer mais vezes simplesmente porque o documento é maior.
Portanto, é necessário normalizar a contagem de termos.

* $\text{TF}_{t, d} = \frac{f_{t, d}}{\lvert d \rvert}$

Sendo $f_{t, d}$ a mesma definição acima e $\lvert d \rvert$ a quantidade total de termos no documento $d$.
Essa é uma métrica normalizada, assim $\sum{TF_{t, d}} = 1$.

* $\text{TF}_{t, d} = \frac{f_{t, d}}{\text{max}_{t'}f_{t', d}}$

Essa definição é similar a anterior, mas ao invés de normalizar pela quantidade total de termos em $d$, a normalização é pela contagem do termo mais frequente $\text{max}_{t'}f_{t', d}$.

A vantagem de normalizar pelo termo mais frequente aparece quando os documentos são muito grandes (contém muitos termos). Veremos um exemplo a seguir.

### Inverse Document Frequency

Apenas as contagens de $\text{TF}$ não representam o conteúdo de um documento, visto que existem palavras que aparecem mais frequentemente.

Portanto, é necessário ponderar a importância de um determinado termo.

Para isso, usa-se

$\text{IDF}_t = \text{log}_2(\frac{N}{n_t})$

Sendo, $N$ a quantidade total de documentos e $n_t$ a quantidade de documentos que o termo $t$ aparece


Assim que tiver calculado os valores de $\text{TF}$ e $\text{IDF}$ é possível calcular o valor final com

$\text{TF-IDF}_{t, d} = \text{TF}_{t, d} \times \text{IDF}_t$


Note que remover palavras muito comuns ( _stop words_ ) pode ser importante antes de calcular o 
$\text{TF-IDF}_{t, d}$ visto que algumas palavras sem importância já foram removidas.

Referências:
* http://www.tfidf.com/
* [Mining Massive Datasets, p. 7-9](http://www.mmds.org/)
* [Introduction to Information Retrieval, p. 117-120](https://nlp.stanford.edu/IR-book/information-retrieval-book.html)
* [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)
* [Clustering Text Documents using k-means](https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#sphx-glr-auto-examples-text-plot-document-clustering-py)
* [TdIdfTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer)

In [14]:
# Alguns documentos para tests
documentos = []

# Fonte: http://www.furb.br/web/1704/noticias/estudantes-participam-da-semana-global-de-empreendedorismo/7989
documentos.append('''Estudantes de Tecnologia em Marketing da FURB, na disciplina Comunicação Empresarial, são os responsáveis pela comunicação local da Semana Global de Empreendedorismo. O evento, criado em 2007, tem como objetivo fortalecer e disseminar a cultura empreendedora e este ano ocorre de 18 a 24 de novembro, em mais de 170 países. A expectativa global é envolver mais de 10 milhões de pessoas em 35 mil eventos cadastrados.
 
Em Blumenau a programação acontece nesta segunda, 18 e terça, 19 de novembro, no auditório da Biblioteca Universitária, campus 1 da FURB. “O evento é público, mas estamos direcionando aos estudantes da Universidade, começando por envolvê-los na divulgação local”, explica a professora de Comunicação Empresarial, Heloísa Rosa. Além dela, estão na organização local da Semana Global de Empreendedorismo os professores Camila Schmitt, Silvio Luís de Vasconcellos e Valter Augusto Krauss, todos ligados ao Centro de Ciências Sociais Aplicadas (CCSA) da FURB.
 
As atividades iniciam nesta segunda-feira, 18, às 18h30, com palestra de Ionita Lunelli, analista do Sebrae/SC há mais de 20 anos. Ela fala sobre o Sebrae e o estímulo que promove ao empreendedorismo. Na sequência, o consultor para o desenvolvimento de startups do Instituto Nexxera | NexxLabs e criador da Metodologia TCC Start Up, João Geraldo Campos, apresenta Negócios com Propósito: Cases de Empreendedorismo Social. O convidado tem formação em Educação Empreendedora pela Babson College, Massachusetts e foi o vencedor do Prêmio Nacional de Educação Empreendedora ENDEAVOR Brasil 2016, com o projeto TCC Start Up, vinculado à Agência de Inovação e Empreendedorismo da Unisul (AGETEC).
 
Na terça-feira, 19, também a partir das 18h30, a FURB recebe a equipe da Haco Etiquetas: Bruno Brandão (Gerente de Marketing), Silvio Triques (Coordenador de Marketing), Taís Prawucki (Planejamento e Criação) e Osiris Reis (Estratégias de Marketing Digital e Relacionamento) para a palestra Marketing 360º.
 
Nos dois dias as palestras ocorrem no Auditório da Biblioteca Universitária, campus 1 da FURB, das 18h30 às 22h, a participação é gratuita e as inscrições podem ser feitas neste link.''')

# Fonte: http://www.furb.br/web/1704/noticias/jornada-reune-especialistas-em-citopatologia-veterinaria/7988
documentos.append('''Nos dias 22 e 23 de novembro, o auditório do Bloco J, campus 1 da FURB, será movimentado por uma série de apresentações e palestras que abordarão temas como o combate e detecção de doenças em animais como cães e gatos, por exemplo. A primeira edição da Jornada de Citopatologia Veterinária está sendo organizada pelo Grupo de Estudos em Patologia Veterinária da FURB, o GEPAV. A Jornada é voltada a acadêmicos, professores e profissionais com interesse na área da citologia, ou seja, o estudo do desenvolvimento e das funções das células nos animais.
 
Conforme a professora do curso de Medicina Veterinária da FURB, Joelma Lucioli, que auxilia na organização do evento, “o objetivo é fornecer informações atualizadas e de qualidade, além de promover o intercâmbio técnico e científico entre o público envolvido neste conjunto de palestras e apresentações, contando com a participação de profissionais com ampla experiência em citopatologia”. Entre os especialistas que estarão presentes na jornada acadêmica estão o Dr. Breno Souza Salgado (UFES/ES), a Dra. Lidiane Narducci Monteiro (CODIVET/ES) e o mestre Leonardo Dourado da Costa (Instituto Qualittas).
 
O evento tem início na sexta-feira, dia 22 de novembro, com a retirada de materiais e abertura oficial, que acontece a partir das 18h no auditório do Bloco J, campus 1. Às 19h, terá início palestra que discutirá a importância da citologia na conduta clínica, destacando erros e limitações. Em seguida, às 20h40, outro painel tratará da coleta e do processamento das amostras citológicas. Haverá um intervalo de 20 minutos entre uma palestra e outra.
 
Já o sábado, 23, será um dia inteiro de apresentações e trocas de conhecimentos entre os profissionais envolvidos. A programação tem início às 8h da manhã, com a continuação do painel que aborda as amostras citológicas, trazendo as interpretações de resultados citológicos aferidos. Às 10h, outra apresentação irá explanar sobre o diagnóstico citológico de neoplasias de células redondas. Em seguida, às 11h30, haverá uma roda de conversas que irá discutir casos clínicos observados no cotidiano por profissionais da área.
 
Durante a tarde, a partir das 14h, acontecerá palestra sobre o diagnóstico citológico das principais neoplasias epiteliais, que acometem a pele dos animais. Às 16h, outra palestra irá trazer os estudos sobre as principais doenças infecciosas de pele em cães e gatos. Logo após, haverá roda de discussão de casos clínicos observados por profissionais e os diagnósticos obtidos. Às 18 horas está marcada a cerimônia de encerramento.
 
Para participar da Jornada de Citopatologia Veterinária, as inscrições deverão ser feitas através deste link até o dia 20 de novembro.
 
O valor da inscrição para estudantes é de R$ 50,00 e para profissionais da área, R$ 100,00.
 
Sócios da Associação Brasileira de Patologias Veterinárias (ABPV), terão 10% de desconto na inscrição. O pagamento pode ser feito através de depósito ou transferência bancária.
 
Mais informações podem ser obtidas pelo e-mail gepavfurb@gmail.com.
 
A 1ª Jornada Acadêmica de Patologia Veterinária é uma realização do Grupo de Estudos em Patologia Veterinária da FURB (GEPAV), com o apoio da Associação Brasileira de Patologia Veterinária (ABPV) e auxílio financeiro da Fundação de Amparo à Pesquisa do Estado de Santa Catarina (FAPESC), através da Chamada Pública 01/2019.''')

# Fonte: http://www.furb.br/web/1704/noticias/pelo-terceiro-mes-seguido-variacao-de-precos-negativa/7984
documentos.append('''O Índice de Variação Geral de Preços do município de Blumenau (IVGP) calculado pela FURB para o mês de outubro apontou uma redução de preços em 92, dos 511 itens pesquisados para compor o índice. Organizados em 25 subgrupos, no último mês, cinco subgrupos registraram alta, enquanto 12 permaneceram estáveis e oito variaram negativamente.
 
Os produtos de limpeza registraram a maior baixa de preços: -3,55%, seguidos dos alimentos industrializados que variaram -1,68% e dos serviços públicos, -1,10%.
Já os itens que registraram a maior alta de preços foram os materiais de construção, +2,38%, os alimentos semi-industrializados, +1,25% e as autopeças, + 1,13%.  
Assim, a variação de preços no mês de outubro foi negativa, -0,29%, abaixo do intervalo esperado pelos economistas, que era de 0,00% e +0,50%. A variação acumulada nos últimos doze meses também está negativa, em -0,09%.
 
É o terceiro mês seguido que o IVGP registra variação negativa: em agosto foi de -0,93%, em setembro, -0,67% e em outubro, -0,29%. No mesmo mês do ano passado, o IVGP registrava +0,59%, maior índice dos últimos 12 meses.
 
Mantido o atual contexto, a variação geral de preços acumulada nos 12 meses, até dezembro de 2019, tende a configurar o cenário otimista, isto é, +3,66%, com uma variação média de 0,30% ao mês. 
 
Para acompanhar a série histórica do índice, calculado desde 1993, basta acessar www.furb.br/ivgp''')

In [30]:
import string

docs_termos = []
for doc in documentos:
    termos = word_tokenize(doc)
    termos = (t for t in termos if t not in sws)
    termos = (t for t in termos if len(t) > 1)
    termos = (t for t in termos if t not in string.punctuation)
    termos = (t.lower() for t in termos)
    termos = list(termos)
    docs_termos.append(termos)


In [47]:
from collections import Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# também é possível usar
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

dv = DictVectorizer()
X = dv.fit_transform(Counter(d) for d in docs_termos)

In [48]:
X[0, :].toarray()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 1., 3., 0., 3., 2., 0., 0., 0., 1.,
        1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1.,
        0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0.,
        0., 1., 0., 0., 1., 0., 0., 0., 2., 1., 0., 0., 0., 1., 0., 0.,
        0., 2., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 2., 1., 0.,
        0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 1., 0., 1., 0., 3., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
        1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 2., 1., 1., 3.,
        5., 2., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
        0., 1., 2., 0., 0., 0., 1., 1., 2., 1., 0., 1., 0., 0., 

In [52]:
# termos no documento
mask = X[0, :].toarray().flatten() > 0
np.array(dv.get_feature_names())[mask]

array(['10', '170', '18', '18h30', '19', '20', '2007', '2016', '22h',
       '24', '35', '360º', 'acontece', 'agetec', 'agência', 'além',
       'analista', 'ano', 'anos', 'aplicadas', 'apresenta', 'as',
       'atividades', 'auditório', 'augusto', 'babson', 'biblioteca',
       'blumenau', 'brandão', 'brasil', 'bruno', 'cadastrados', 'camila',
       'campos', 'campus', 'cases', 'ccsa', 'centro', 'ciências',
       'college', 'começando', 'comunicação', 'consultor', 'convidado',
       'coordenador', 'criado', 'criador', 'criação', 'cultura',
       'desenvolvimento', 'dias', 'digital', 'direcionando', 'disciplina',
       'disseminar', 'divulgação', 'dois', 'educação', 'ela', 'em',
       'empreendedora', 'empreendedorismo', 'empresarial', 'endeavor',
       'envolver', 'envolvê-los', 'equipe', 'estratégias', 'estudantes',
       'estímulo', 'etiquetas', 'evento', 'eventos', 'expectativa',
       'explica', 'fala', 'feitas', 'formação', 'fortalecer', 'furb',
       'geraldo', 'gerent

In [53]:
# top-k termos
k = 10
args = np.argsort(X[0, :].toarray().flatten())
np.array(dv.get_feature_names())[args[:k]]

array(['+0,50', 'já', 'jornada', 'joelma', 'ivgp', 'itens', 'irá',
       'início', 'intervalo', 'interpretações'], dtype='<U21')

In [54]:
tfidf = TfidfTransformer(norm='l1')  # l1 norm, divide pela quantidade de termos no documento
X = tfidf.fit_transform(X)

In [55]:
# top-k termos
k = 10
args = np.argsort(X[0, :].toarray().flatten())
np.array(dv.get_feature_names())[args[:k]]

array(['+0,50', 'já', 'jornada', 'joelma', 'ivgp', 'itens', 'irá',
       'início', 'intervalo', 'interpretações'], dtype='<U21')

## Part of Speech (POS) tagging

Anota as palavras de acordo com a sua função linguística na frase.
Ou seja, indica se uma palavra é um verbo, substantivo, pontuação, outros.

In [55]:
lemmas = [t.pos_ for t in spacy_tokens_sem_stopwords]
lemmas[:10]

['DET',
 'PROPN',
 'VERB',
 'ADJ',
 'PROPN',
 'PROPN',
 'PROPN',
 'PUNCT',
 'DET',
 'NOUN']

In [56]:
# pegando apenas os verbos
for token in filter(lambda x: x.pos_ == 'VERB', doc):
    print(token)

desponta
top
seguida
está
seguida


In [57]:
# pegando apenas os substantivos
for token in filter(lambda x: x.pos_ == 'NOUN', doc):
    print(token)

total
país
lugar
Pontifícia


In [59]:
# pegando apenas os nomes próprios
for token in filter(lambda x: x.pos_ == 'PROPN', doc):
    print(token)

Brasil
Universidades
América
Latina
10
do
Chile
Colômbia
México
Argentina
Universidade
Católica
Chile
da
Universidade
São
Paulo
USP


## Named Entity Recognition (NER)

Reconhece e classifica entidades no texto

In [61]:
for entity in doc.ents:
    print(entity.text, entity.label_)

Brasil LOC
Universidades da América Latina PER
Chile LOC
Colômbia LOC
México LOC
Argentina LOC
Pontifícia Universidade Católica do Chile LOC
Universidade de São Paulo LOC
USP LOC
